In [137]:
c0_neg = list(pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster0')['review'])
c1_neg = list(pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster1')['review'])
c2_neg =list( pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster2')['review'])
c3_neg = list(pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster3')['review'])
c4_neg = list(pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster4')['review'])
c5_neg = list(pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster5')['review'])
c6_neg = list(pd.read_excel('negative_cluster_more.xlsx',sheet_name='cluster6')['review'])
c0_pos = list(pd.read_excel('positive_cluster_more.xlsx',sheet_name='cluster0')['review'])
c1_pos =list( pd.read_excel('positive_cluster_more.xlsx',sheet_name='cluster1')['review'])
c2_pos = list(pd.read_excel('positive_cluster_more.xlsx',sheet_name='cluster2')['review'])

In [168]:
def remove_num_name(corpus):
    news=[]
    for i in corpus:
        new=re.sub(r'[0-9]+','',str(i))# remove numbers
        new=new.replace('\n','').replace('\\','').replace('<','').replace('/','').replace('>','').replace('_','').replace('<br />','').replace('(','').replace(')','')          # remove delimeter 
                              
        #new=re.sub(r'[A-Z][a-z]*[\s-][A-Z][a-z]*','',new) # remove full name
        #new=re.sub(r'[A-Z][a-z]+','',new)                 # remove part of name
        news.append(new)
    return news

In [169]:
c0_neg=remove_num_name(c0_neg)
c1_neg=remove_num_name(c1_neg)
c2_neg=remove_num_name(c2_neg)
c3_neg=remove_num_name(c3_neg)
c4_neg=remove_num_name(c4_neg)
c5_neg=remove_num_name(c5_neg)
c6_neg=remove_num_name(c6_neg)
c0_pos=remove_num_name(c0_pos)
c1_pos=remove_num_name(c1_pos)
c2_pos=remove_num_name(c2_pos)

In [267]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english')+['movie','would','really', 'thing','could'])
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = [i for i in nltk.word_tokenize(doc.lower()) if (i not in stop) and (len(i)>4)]
    punc_free = ' '.join(ch for ch in stop_free if ch not in exclude)
    stop_free2=' '.join(word for word in punc_free.split()if (word not in stop) and (len(i)>4))
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    res=' '.join(word for word in normalized.split()if (word not in stop) and (len(i)>4))
    return res


In [268]:
for i in ['c0_neg','c1_neg','c2_neg','c3_neg','c4_neg','c5_neg','c6_neg','c0_pos','c1_pos','c2_pos']:
    exec('clean_{} = [clean(doc).split() for doc in {}]'.format(i,i))

<h3> LDA

In [273]:
import numpy as np
from gensim import corpora, models, similarities 
#create a Gensim dictionary from the texts
for tokens in [clean_c0_neg,clean_c1_neg,clean_c2_neg,clean_c3_neg,clean_c4_neg,clean_c5_neg,clean_c6_neg,clean_c0_pos,clean_c1_pos,clean_c2_pos]:
    dictionary = corpora.Dictionary(tokens)

    #remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
    dictionary.filter_extremes(no_below=1, no_above=0.8)

    #convert the dictionary to a bag of words corpus for reference
    corpus = [dictionary.doc2bow(text) for text in tokens]
    %time lda = models.LdaModel(corpus, num_topics=3,id2word=dictionary,\
                                update_every=5,\
                                chunksize=600,\
                                passes=100)
    topics_matrix = lda.show_topics(formatted=False, num_words=20)
    topics_matrix=[i[1] for i in topics_matrix]
    topics_matrix = np.array(topics_matrix)

    topic_words = topics_matrix[:,:,0]
    for i in topic_words:
        print([str(word) for word in i])
        print()

Wall time: 18min 54s
['american', 'world', 'people', 'political', 'documentary', 'propaganda', 'dream', 'stone', 'german', 'soldier', 'london', 'salman', 'audience', 'british', 'black', 'family', 'seberg', 'japanese', 'history', 'niven']

['character', 'story', 'woman', 'performance', 'novel', 'actor', 'version', 'seems', 'scene', 'played', 'young', 'director', 'play', 'great', 'rather', 'year', 'taylor', 'musical', 'richard', 'look']

['character', 'scene', 'story', 'people', 'first', 'think', 'acting', 'watch', 'actor', 'better', 'little', 'never', 'nothing', 'something', 'film', 'watching', 'minute', 'actually', 'going', 'great']

Wall time: 25.7 s
['people', 'scene', 'flick', 'film', 'horror', 'make', 'story', 'acting', 'infected', 'group', 'woman', 'character', 'military', 'death', 'revolt', 'first', 'fulci', 'watching', 'worst', 'ireland']

['people', 'still', 'acting', 'vampire', 'effect', 'think', 'story', 'louque', 'nothing', 'never', 'scene', 'woman', 'living', 'better', 'who